## KeyPhrase extraction
* Use SemEval 2010 dataset - [H49](https://github.com/boudinfl/ake-datasets/blob/master/datasets/SemEval-2010/train/H-49.xml) dataset for TF-IDF vectorization
* Use SemEval 2010 dataset - [H7](https://github.com/boudinfl/ake-datasets/blob/master/datasets/SemEval-2010/test/H-7.xml) for inference

In [1]:
import operator
import json
import numpy as np
from pathlib import Path
from glob import glob
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from xml.etree import ElementTree